In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
Path.cwd().parent / 'src'

WindowsPath('C:/Users/r2d4/OneDrive/code/projects/20-02_disco/src')

In [4]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import (
    Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, CreditTrx, LabelRecordLink, ArtistLabelLink
)
from src import db_functions
from src import db_connect

In [5]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To DB

Because we connect from `dev`subfolder, I have to work with the sqlalchemy `create_engine` function directly.

In [6]:
# session.close()
prod_path = r"sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase.db"
engine = sqlalchemy.create_engine(prod_path)

session = db_connect.create_session(engine)

# Check
session

## Query DB

In [7]:
pd.read_sql("records", engine).tail(5)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
310,311,Most Dangerous Game,2020,5,6,clear with red splatter,400,None,None,2021-02-19,10,None,1,1,2021-02-24 14:24:26,NaT
311,312,Loyalty At All Costs,2021,5,6,black with red swirl,100,None,Streets Of Hate exclusive,2021-02-19,10,None,0,1,2021-02-24 14:24:26,2021-02-28 12:14:09
312,313,Ultimate Aggression,2019,5,3,"half baby blue, half electric blue",300,None,None,2020-09-13,35,None,1,1,2021-02-24 14:24:26,NaT
313,314,Follow The Blind,1989,4,3,None,None,None,None,2014-02-01,45,None,0,1,2021-02-28 11:28:28,NaT
314,315,Crime And Punishment,2021,5,3,white with red splatter,250,None,None,2021-03-01,25,None,0,1,2021-03-01 14:42:58,NaT


In [8]:
pd.read_sql("records", engine)["is_active"].sum()

291

In [9]:
pd.read_sql("artists", engine).tail

<bound method NDFrame.tail of      artist_id              artist_name artist_country          created_at          updated_at
0            1                Dismember         Sweden 2021-02-24 14:24:16                 NaT
1            2                Merciless         Sweden 2021-02-24 14:24:16 2021-02-28 16:57:22
2            3                 Overkill            USA 2021-02-24 14:24:16 2021-02-24 14:44:37
3            4                 Entombed         Sweden 2021-02-24 14:24:16                 NaT
4            5                Metallica            USA 2021-02-24 14:24:16 2021-02-28 16:59:06
..         ...                      ...            ...                 ...                 ...
199        200                 Fleshrot            USA 2021-02-24 14:24:26                 NaT
200        201                   End It            USA 2021-02-24 14:24:26                 NaT
201        202                   Kharma            USA 2021-02-24 14:24:26                 NaT
202        203      

In [10]:
pd.read_sql("credit_trx", engine).tail(10)

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
316,317,2021-02-08,Purchase,-1.00,-5.00,308.00,2021-02-24 14:24:26,NaT
317,318,2021-02-18,Addition,1.00,-4.00,nan,2021-02-24 14:24:26,NaT
318,319,2020-10-23,Purchase,0.00,-4.00,309.00,2021-02-24 14:24:26,NaT
319,320,2021-02-19,Purchase,0.00,-4.00,311.00,2021-02-24 14:24:26,NaT
320,321,2021-02-19,Purchase,0.00,-4.00,312.00,2021-02-24 14:24:26,NaT
321,322,2021-02-19,Purchase,0.00,-4.00,310.00,2021-02-24 14:24:26,NaT
322,323,2020-09-13,Purchase,0.00,-4.00,313.00,2021-02-24 14:24:26,NaT
323,324,2021-02-28,Addition,1.00,-3.00,nan,2021-02-28 10:36:07,NaT
324,325,2014-02-01,Purchase,0.00,-3.00,314.00,2021-02-28 11:28:28,NaT
325,326,2021-03-01,Purchase,-1.00,-4.00,315.00,2021-03-01 14:42:58,NaT


In [11]:
pd.read_sql("record_label_link", engine).tail()
pd.read_sql("artist_label_link", engine).tail()
pd.read_sql("artist_genre_link", engine).tail()
pd.read_sql("genre_label_link", engine).tail()

,record_id,label_id
303,304,25
304,305,33
305,306,33
306,307,33
307,308,45


,artist_id,label_id
292,118,99
293,204,40
294,31,64
295,100,100
296,131,100


,artist_id,genre_id
202,200,1
203,201,5
204,202,5
205,203,5
206,204,5


,genre_id,label_id
107,6,96
108,6,97
109,6,98
110,6,99
111,5,100


In [12]:
pd.read_sql("labels", engine).sort_values("label_name")

,label_id,label_name,created_at,updated_at
17,18,20 Buck Spin,2021-02-24 14:24:22,NaT
27,28,6131,2021-02-24 14:24:23,NaT
89,90,Abfall,2021-02-28 17:49:06,NaT
91,92,Acouphènes!,2021-02-28 17:50:46,NaT
9,10,Active Rebellion,2021-02-24 14:24:20,NaT
...,...,...,...,...
49,50,Vinyl Collectors,2021-02-24 16:31:31,NaT
73,74,Virgin,2021-02-28 11:24:25,NaT
69,70,War Anthem,2021-02-28 10:48:59,NaT
64,65,Witching Hour Productions,2021-02-26 17:07:08,NaT


In [13]:
result = session.query(Record).filter(Record.artists.any(Artist.artist_name == "Entombed")).all()
for record in result:
    print(record, record.purchase_date)

<Record(record_id=11, title=Uprising, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2013-07-01
<Record(record_id=12, title=Morning Star, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2013-07-01
<Record(record_id=23, title=Wolverine Blues, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2014-01-01
<Record(record_id=36, title=Clandestine, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2015-08-01
<Record(record_id=40, title=Left Hand Path, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2015-10-01


C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [14]:
result = session.query(Artist).filter(Artist.artist_name == "Entombed").one()
print(result.artist_country)

Sweden


In [15]:
pd.read_sql("formats", engine)

,format_id,format_name,created_at,updated_at
0,1,"12""",2021-02-24 14:24:16,NaT
1,2,"Pic-7""",2021-02-24 14:24:16,2021-02-24 14:37:30
2,3,LP,2021-02-24 14:24:16,NaT
3,4,2LP,2021-02-24 14:24:16,NaT
4,5,"10""",2021-02-24 14:24:16,NaT
5,6,"7""",2021-02-24 14:24:16,NaT
6,7,MLP,2021-02-24 14:24:17,NaT
7,8,2xLP,2021-02-24 14:24:17,NaT
8,9,LP,2021-02-24 14:24:20,NaT
9,10,Pic-LP,2021-02-24 14:24:21,NaT


In [16]:
pd.read_sql("genres", engine)

,genre_id,genre_name,created_at,updated_at
0,1,Death Metal,2021-02-24 14:24:16,NaT
1,2,Thrash Metal,2021-02-24 14:24:16,NaT
2,3,Black Metal,2021-02-24 14:24:16,NaT
3,4,Speed Metal,2021-02-24 14:24:17,NaT
4,5,Hardcore,2021-02-24 14:24:18,NaT
5,6,Punk,2021-02-24 14:24:19,NaT
6,7,Crust,2021-02-24 14:24:19,NaT
7,8,Crossover,2021-02-24 14:24:24,NaT
8,9,Grindcore,2021-02-24 14:24:25,NaT


## Inactivate A Record (Temporary Section)

In [17]:
# removal = {
#     "trx_type": "Removal",
#     "credit_value": 0,
#     "artist": "Furia",
#     "title": "Marzannie, Królowej Polski",
#     "year": 2012,
#     "date": dt.datetime.today().date()
# }

In [18]:
# db_functions.set_record_to_inactive(session, removal)

In [19]:
# pd.read_sql("credit_trx", engine).tail()

In [20]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 301).one()
# session.delete(to_delete)
# session.commit()

## Backfill Missing Record-Format-Relations in Records Table (Done)

In [21]:
# def load_albums_from_xlsx(
#     filepath: Union[Path, str], genres: Optional[List] = None
# ) -> pd.DataFrame:
#     """Load the original album collection file into a dataframe.
#     You can specify a list of genres you want to include
#     (defaults to None).
#     """
#     df = pd.read_excel(filepath, engine="openpyxl")
#     if genres:
#         df = df[df["Genre"].isin(genres)]
#     return df

In [22]:
# CONFIG_PATH = Path.cwd().parent / "config.cfg"
# path_to_collection = utils.read_config_return_str(CONFIG_PATH, "COLLECTION")
# collection_df = load_albums_from_xlsx(path_to_collection)
# collection_df = clean.prettify_column_names(collection_df)
# collection_df.dropna(thresh=8, inplace=True)
# collection_df.rename(columns={"format": "record_format"}, inplace=True)
# # collection_df["purchase_date"] = collection_df["purchase_date"].dt.date.astype(str)
# # collection_df["number"] = collection_df["number"].astype(str)
# # collection_df["digitized"] = collection_df["digitized"].map({True: 1, False: 0}).astype(int)
# collection_df["label"].fillna("NA", inplace=True)  # because it creates a foreign key

# # This is for the initial ingestion only
# collection_df["artist_country"] = None
# collection_df["credit_value"] = 0
# collection_df["trx_type"] = "Initial Load"

# assert len(collection_df) == 293

In [23]:
# # backfill

# for rec in collection_df.itertuples():
#     record = db_functions.fetch_a_record_from_the_shelf(session, rec.artist, rec.title)
#     r_format = rec.record_format
    
#     # Check if the format already exists or has to be created
#     record_format = (
#         session.query(RecordFormat)
#         .filter(RecordFormat.format_name.ilike(r_format))
#         .one_or_none()
#     )
#     if record_format is None:
#         record_format = RecordFormat(format_name=r_format)
#         session.add(record_format)
    
#     record.record_format = record_format

# session.commit()

In [24]:
# # Manual Update

# record = session.query(Record).filter(Record.record_id == 295).one()
# r_format = '12"'

# # Check if the format already exists or has to be created
# record_format = (
#     session.query(RecordFormat)
#     .filter(RecordFormat.format_name.ilike(r_format))
#     .one_or_none()
# )
# if record_format is None:
#     record_format = RecordFormat(format_name=r_format)
#     session.add(record_format)

# record.record_format = record_format

# session.commit()

## Manual Updates

### Credit Trx

In [25]:
# result = session.query(CreditTrx).filter(CreditTrx.credit_trx_type == "Removal").all()
# for r in result:
#     r.credit_trx_type = "Remove"
#     print(r)

In [26]:
# r = session.query(Artist).filter(Artist.artist_name == "CELESTIAL SANCTUARY").one()
# r.artist_name = "Celestial Sanctuary"

In [27]:
# session.commit()

### Remove "Records" From Label-Names And Split Labels

In [28]:
# result = session.query(Label).filter(Label.label_name.like('% Records')).all()
# for x in result:
#     x.label_name = x.label_name.rpartition(" Records")[0]
# session.commit()

In [29]:
# session.query(Label).get(42)

In [30]:
# result = session.query(Label).filter(Label.label_name.contains(' / ')).all()
# for x in result:
#     print(x)

In [31]:
# result = session.query(Record).filter(Record.labels.any(Label.label_id.in_([19, 31, 32])))
# for x in result:
#     print(x)

In [32]:
# jack = session.query(Label).filter(Label.label_id.in_([19, 31, 32])).all()
# for x in jack:
#     print(x)
#     session.delete(x)
# session.commit()

In [33]:
# session.close()

### Update Stuff

In [54]:
result = session.query(Record).filter(Record.artists.any(Artist.artist_country == "NA")).first()
result



<Record(record_id=220, title=Death Velour, artist=[<Artist(artist_id=134, artist_name=Ghastly)>])>

In [40]:
# result = session.query(Artist).filter(Artist.artist_name == "Insurgency").first()
# result.artist_name = "Resurgency"
# session.commit()

In [ ]:
# session.close()
# session = db_connect.create_session(engine)